In [2]:
from export import *

In [4]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [5]:
def get_data():
    mnist_url='http://deeplearning.net/data/mnist/mnist.pkl'
    path = datasets.download_data(mnist_url,ext='.gz')
    with gzip.open(path,'rb') as f:
        ((x_train, y_train), (x_valid,y_valid), _) = pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train,y_train,x_valid,y_valid))
def normalize(x,m,s):
    return (x-m)/s

In [6]:
x_train,y_train,x_valid,y_valid = get_data()
y_train = y_train.float()
y_valid = y_valid.float()
m,s = x_train.mean(),x_train.std()
x_train = normalize(x_train,m,s)
x_valid = normalize(x_valid,m,s)
n,c = x_train.shape
n,c,m,s

In [11]:
class Relu():
    def __call__(self,inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g
        
class Linear():
    def __init__(self,w,b):
        self.w = w
        self.b = b
        
    def __call__(self,inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g =  self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        #self.w.g = torch.einsum("ki,kj->ij",self.inp,self.out.g)
        #self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

class Mse():
    def __call__(self,inp,targ):
        self.inp = inp
        self.targ = targ      
        
        self.out = (self.inp.squeeze(-1) - self.targ).pow(2).mean()
        return self.out
    
    def backward(self):        
        self.inp.g = 2 * (self.inp.squeeze(-1) - self.targ).unsqueeze(-1) / self.targ.shape[0]
        
        
class Model():
    def __init__(self,w1,b1,w2,b2):
        self.layers = [Linear(w1,b1),Relu(),Linear(w2,b2)]
        self.loss = Mse()
        
    def __call__(self,x,targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()
        

In [12]:
nh = 50
#w1 = torch.randn(c,nh)*math.sqrt(2/c)
w1=torch.zeros(c,nh)
torch.nn.init.kaiming_normal_(w1,mode='fan_out')
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)*math.sqrt(2/nh)
torch.nn.init.kaiming_normal_(w2,mode='fan_out')
b2 = torch.zeros(1)

In [13]:
model = Model(w1,b1,w2,b2)

In [14]:
%time loss = model(x_train,y_train.float())

CPU times: user 247 ms, sys: 5.08 ms, total: 252 ms
Wall time: 228 ms


In [15]:
%time model.backward()

CPU times: user 296 ms, sys: 113 ms, total: 409 ms
Wall time: 181 ms
